In [3]:
print("Hello, test!")

Hello, test!


In [1]:
%pwd

'/Users/belalmohsen/Breast-cancer/RafeekBot/research'

In [2]:
import os
os.chdir("../")

In [3]:
%pwd

'/Users/belalmohsen/Breast-cancer/RafeekBot'

In [6]:
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [10]:
data_path = "/Users/belalmohsen/Breast-cancer/data"
def load_pdf_files(data_path):
    loader=DirectoryLoader(
        data_path,
        glob="*.pdf",
        loader_cls=PyPDFLoader
    )

    documents=loader.load()
    return documents


In [ ]:

extracted_data = load_pdf_files(data_path)

In [12]:
extracted_data

[Document(metadata={'producer': 'Adobe PDF Library 15.0', 'creator': 'Adobe InDesign CC 13.0 (Macintosh)', 'creationdate': '2018-03-11T14:29:06-04:00', 'author': 'American Society of Clinical Oncology (ASCO)', 'moddate': '2018-05-17T13:54:55-04:00', 'title': 'ASCO Answers: Guide to Breast Cancer', 'trapped': '/False', 'source': '/Users/belalmohsen/Breast-cancer/data/breast-cancer-answers-american-society-of-clinical-oncology-asco.pdf', 'total_pages': 50, 'page': 0, 'page_label': 'C1'}, page_content='Breast \nCancer \nTrusted Information to Help Manage Your Care from \nthe American Society of Clinical Oncology'),
 Document(metadata={'producer': 'Adobe PDF Library 15.0', 'creator': 'Adobe InDesign CC 13.0 (Macintosh)', 'creationdate': '2018-03-11T14:29:06-04:00', 'author': 'American Society of Clinical Oncology (ASCO)', 'moddate': '2018-05-17T13:54:55-04:00', 'title': 'ASCO Answers: Guide to Breast Cancer', 'trapped': '/False', 'source': '/Users/belalmohsen/Breast-cancer/data/breast-canc

In [13]:
len(extracted_data)

206

In [17]:
from typing import List
from langchain.schema import Document

def filter_to_minimal_docs(docs: List[Document]) -> List[Document]:
    """
    Given a list of Document objects, return a new list of Document objects
    containing only 'source' in metadata and the original page_content.
    """
    minimal_docs: List[Document] = []
    for doc in docs:
        src = doc.metadata.get("source")
        minimal_docs.append(
            Document(
                page_content=doc.page_content,
                metadata={"source": src}
            )
        )
    return minimal_docs

In [18]:
minimal_docs = filter_to_minimal_docs(extracted_data)

In [19]:
minimal_docs

[Document(metadata={'source': '/Users/belalmohsen/Breast-cancer/data/breast-cancer-answers-american-society-of-clinical-oncology-asco.pdf'}, page_content='Breast \nCancer \nTrusted Information to Help Manage Your Care from \nthe American Society of Clinical Oncology'),
 Document(metadata={'source': '/Users/belalmohsen/Breast-cancer/data/breast-cancer-answers-american-society-of-clinical-oncology-asco.pdf'}, page_content='-  \n \n \n \n  \n \n- -\n \n  \n  \nABOUT ASCO \nFounded in 1964, the American Society of Clinical Oncology, Inc. (ASCO®) is committed to making a world of difference \nin cancer care. As the world’s leading organization of its kind, ASCO represents nearly 45,000 oncology professionals who \ncare for people living with cancer. Through research, education, and promotion of the highest quality patient care, ASCO \nworks to conquer cancer and create a world where cancer is prevented or cured, and every survivor is healthy. ASCO is \nsupported by its affiliate organizatio

In [20]:
# sploit the documents into smaller chunks
def text_split(minimal_docs):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=20,
    )
    texts_chunk = text_splitter.split_documents(minimal_docs)
    return texts_chunk

In [21]:
texts_splits = text_split(minimal_docs)
print(f"Number of text chunks: {len(texts_splits)}")

Number of text chunks: 930


In [22]:
texts_splits

[Document(metadata={'source': '/Users/belalmohsen/Breast-cancer/data/breast-cancer-answers-american-society-of-clinical-oncology-asco.pdf'}, page_content='Breast \nCancer \nTrusted Information to Help Manage Your Care from \nthe American Society of Clinical Oncology'),
 Document(metadata={'source': '/Users/belalmohsen/Breast-cancer/data/breast-cancer-answers-american-society-of-clinical-oncology-asco.pdf'}, page_content='-  \n \n \n \n  \n \n- -\n \n  \n  \nABOUT ASCO \nFounded in 1964, the American Society of Clinical Oncology, Inc. (ASCO®) is committed to making a world of difference \nin cancer care. As the world’s leading organization of its kind, ASCO represents nearly 45,000 oncology professionals who \ncare for people living with cancer. Through research, education, and promotion of the highest quality patient care, ASCO'),
 Document(metadata={'source': '/Users/belalmohsen/Breast-cancer/data/breast-cancer-answers-american-society-of-clinical-oncology-asco.pdf'}, page_content='wo

In [23]:
from langchain.embeddings import HuggingFaceEmbeddings
def download_embeddings():
    """
    Download and return the HuggingFace embeddings model.
    """
    model_name = "sentence-transformers/all-MiniLM-L6-v2"
    embeddings = HuggingFaceEmbeddings(
        model_name=model_name
    )
    return embeddings

embedding = download_embeddings()

/var/folders/gb/3gd6t2dn1r30w_t1ld_0b9_r0000gn/T/ipykernel_3901/2883842866.py:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


In [24]:
embedding

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [25]:
vector =embedding.embed_query("Hello world")
vector

[-0.03447727859020233,
 0.031023172661662102,
 0.006734957918524742,
 0.026109017431735992,
 -0.03936202824115753,
 -0.16030247509479523,
 0.06692398339509964,
 -0.006441447418183088,
 -0.04745051637291908,
 0.014758891426026821,
 0.07087536156177521,
 0.05552758648991585,
 0.019193295389413834,
 -0.02625131607055664,
 -0.010109537281095982,
 -0.026940515264868736,
 0.02230743318796158,
 -0.022226635366678238,
 -0.1496926248073578,
 -0.017493057996034622,
 0.007676220033317804,
 0.054352324455976486,
 0.003254470182582736,
 0.031725961714982986,
 -0.08462142944335938,
 -0.029405996203422546,
 0.05159565806388855,
 0.0481240339577198,
 -0.003314804285764694,
 -0.058279190212488174,
 0.04196927696466446,
 0.02221071906387806,
 0.12818880379199982,
 -0.022338921204209328,
 -0.011656297370791435,
 0.06292841583490372,
 -0.03287629783153534,
 -0.09122607856988907,
 -0.031175291165709496,
 0.052699558436870575,
 0.04703482985496521,
 -0.08420306444168091,
 -0.030056182295084,
 -0.02074482850

In [26]:
print(len(vector))

384


In [51]:
from dotenv import load_dotenv
load_dotenv()
pinecone_api_key = os.getenv("PINECONE_API_KEY")
gemini_api_key = os.getenv("GEMINI_API_KEY")


os.environ["PINECONE_API_KEY"] = pinecone_api_key
os.environ["GEMINI_API_KEY"] = gemini_api_key

In [52]:
from pinecone import Pinecone
pc = Pinecone(api_key=pinecone_api_key)

In [53]:
pc

In [54]:
from pinecone import ServerlessSpec 

index_name = "rafeek-bot"

if not pc.has_index(index_name):
    pc.create_index(
        name = index_name,
        dimension=384,  # Dimension of the embeddings
        metric= "cosine",  # Cosine similarity
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )


index = pc.Index(index_name)

In [56]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=texts_splits,
    embedding=embedding,
    index_name=index_name,
)

In [57]:
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embedding
)

# add more metadata to the documents   

In [ ]:
dswith = Document(
    page_content="dswithbappy is a youtube channel that provides tutorials on various topics.",
    metadata={"source": "Youtube"}
)

In [ ]:
docsearch.add_documents(documents=[dswith])


#

In [58]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [59]:
retrieverdocs = retriever.invoke("What is grade?")

In [60]:
retrieverdocs

[Document(id='1d6153d9-88cf-44f1-babc-28d093176587', metadata={'source': '/Users/belalmohsen/Breast-cancer/data/breast-cancer-answers-american-society-of-clinical-oncology-asco.pdf'}, page_content='12 \nASCO Answers\n \n \n \n \n \n \n \n \n \n \n \n  \n  \n  \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n  \n  \n  \n \n  \n  \n  \n \n \n \n  \n \n \n \nGrade \nDoctors may also use the term “grade” when talking about \nbreast cancer. The grade describes how much cancer cells \nlook like healthy cells when viewed under a microscope. \nKnowing the grade of the cancer may help your doctor \npredict how quickly the cancer will spread. If the cancer looks \nsimilar to healthy tissue and contains different cell groupings, it'),
 Document(id='8c4aa7ff-a59d-4b04-8e5d-3d7ba998157e', metadata={'source': '/Users/belalmohsen/Breast-cancer/data/EN_|_Breast_Cancer-_Guide_for_Patients.pdf'}, page_content='GRADE\nCancer grade  is based on how different tumour  cells \nlook from normal c

In [62]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-3-flash-preview", 
    temperature=0.7,
    google_api_key=gemini_api_key
)

In [63]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [82]:
system_template = """
You are "Rafeek" (رفيق), a compassionate and knowledgeable AI medical assistant dedicated to supporting breast cancer patients.

Your Goal: Provide accurate medical information, emotional support, and practical advice based ONLY on the provided context.

Guidelines:
1. **Empathy First:** Always start with a comforting and supportive tone. Use phrases like "I understand," "Stay strong," or "Here to help."
2. **Strict RAG:** Answer the user's question using ONLY the following pieces of retrieved context. Do NOT make up medical facts or use outside knowledge unless it's general common sense (like healthy habits).
3. **Language:** If the user asks in Arabic, answer in Arabic. If in English, answer in English.
4. **Clarity:** Use simple, non-technical language. Break down complex medical terms.
5. **Safety:** If the answer is not in the context, say: "I apologize, but I don't have this specific information in my references. Please consult your doctor for this specific query."
6. **Disclaimer:** Never claim to be a doctor. Always encourage consulting a specialist for critical decisions.

Context:
{context}
"""

prompt= ChatPromptTemplate.from_messages([
    ("system", system_template),
    ("human", "{input}")
])

In [83]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
retrieval_chain = create_retrieval_chain(
    retriever,
    question_answer_chain
)

In [85]:
response = retrieval_chain.invoke({"input": "What is grade?"})
print(response["answer"])

Hello, I’m Rafeek. I understand that hearing medical terms can be overwhelming, but I am here to help you understand them clearly. Stay strong.

Based on the information I have, the **"grade"** of breast cancer describes how much the cancer cells look like healthy, normal cells when viewed under a microscope. 

Here are the key points to understand about the grade:
*   **What it shows:** It reflects how different the tumor cells are from normal cells and how quickly they are growing.
*   **The Scale:** The grade is usually a value between **one and three**.
*   **Aggressiveness:** This value reflects the aggressiveness of the tumor cells. A higher grade means the tumor is more aggressive, while a lower grade means the cells look more similar to healthy tissue.
*   **Prediction:** Knowing the grade helps your doctor predict how quickly the cancer might spread.

Please remember that I am an AI assistant and not a doctor. I recommend discussing your specific grade with your specialist to 